In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
! which python

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

import logging
import locality.utils.logging_utils as logging_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format = logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout
)


In [ ]:
from dsets.counterfact import CounterFactDataset

counterfact = CounterFactDataset(data_dir="../counterfact")

located_in_city = [d for d in counterfact if d['requested_rewrite']['relation_id'] == "P17"]
places_to_cities = [
    (d['requested_rewrite']['subject'], d['requested_rewrite']['target_true']["str"])
    for d in located_in_city
]
print(len(places_to_cities))
places_to_cities[:4]

In [ ]:
import locality.utils.dataset_utils as dset_utils

dset_utils.get_demonstrations(
    subj_obj_mapping=places_to_cities,
    num_options=3,
    num_icl=1,
    variable_binding_template=" {} is visiting {}",
    query_template=" {} is in {}",
)

In [ ]:
from locality.dataset import generate_synthetic_dataset

In [ ]:
synth_dataset = generate_synthetic_dataset(
    relation_subj_obj_mapping=places_to_cities,
    variable_binding_template=" {} is visiting {}",
    query_template=" {} is in {}",
    num_options=3,
    num_icl=5,
    batch_size=32
)

In [ ]:
sample = synth_dataset[30]

print(sample[0])
print(sample[1])

In [ ]:
synth_dataset.qa_samples[10].to_json()

In [ ]:
synth_dataset.to_json()

In [ ]:
places_to_cities[:4]

In [ ]:
from locality.models import ModelandTokenizer

# MODEL_PATH = "EleutherAI/gpt-j-6B"
MODEL_PATH = "meta-llama/Llama-2-7b-hf"
# MODEL_PATH = "mistralai/Mistral-7B-v0.1"

mt = ModelandTokenizer(model_path = MODEL_PATH)

In [ ]:
from locality.functional import make_icl_prompt, filter_samples_by_model_knowledge

known_samples = filter_samples_by_model_knowledge(
    mt, 
    subj_obj_mapping=places_to_cities[:25],
    prompt_template=" {} is located in the country of",
)

In [ ]:
limit = 3

for query, answer in synth_dataset:
    print(query)
    print(answer)
    print()
    limit -= 1
    if limit == 0:
        break

In [ ]:
from locality.functional import get_h

sample = synth_dataset.qa_samples[10]
prompt, subject = sample.query, sample.subject

hrr = get_h(
    mt, 
    prompt, 
    subject,
    layers = [mt.layer_name_format.format(i) for i in range(5, 10)]
)

In [ ]:
for key, val in hrr.items():
    print(key, val.shape)

In [ ]:
mt.tokenizer("Apple")

In [ ]:
import torch

t = torch.randperm(10)
t.topk()

In [ ]:
from locality.functional import predict_next_token

predict_next_token(
    mt, 
    prompt = "Eiffel Tower is located in",
    token_of_interest="France"
)

In [ ]:
arr = [1,2,3]
arr.extend(4)
arr